# Archived Chat Processor

### Import DF

In [18]:
import time
import pandas as pd
pd.set_option('display.max_colwidth', None)

number_of_csv = 39

df = pd.read_csv(f"Archive/archived_threads/39.csv")

df = df.iloc[::-1]

question = {str(df.iloc[0]['author']): df.iloc[0]['content']}

answers = []
for i in range(1, len(df)):
    answers.append({str(df.iloc[i]['author']): df.iloc[i]['content']})

In [5]:
df.describe()

,id,channel_id,author,referenced_message
count,3.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00
mean,1.165160e+18,1.164909e+18,5.111053e+17,1.164909e+18
std,2.603171e+14,0.000000e+00,3.321686e+17,NaN
min,1.165010e+18,1.164909e+18,3.193277e+17,1.164909e+18
25%,1.165010e+18,1.164909e+18,3.193277e+17,1.164909e+18
50%,1.165010e+18,1.164909e+18,3.193277e+17,1.164909e+18
75%,1.165236e+18,1.164909e+18,6.069941e+17,1.164909e+18
max,1.165461e+18,1.164909e+18,8.946606e+17,1.164909e+18


In [79]:
data = [
    question,
    answers
]

In [19]:
import json

file_path = f"processed{number_of_csv}.json"
df.to_json(file_path, index=False)
# json_data = json.dumps(data, indent=2)

# with open(file_path, 'w') as file:
#     file.write(json_data)

# print(f'Data has been exported to {file_path}')

### Assistant

In [2]:
import openai

client = openai.OpenAI(
)
print(openai.VERSION)

1.2.3


In [3]:
assistant = client.beta.assistants.create(
    name="Discord Thread Inferer",
    instructions="""
         You have csv file named processed1.csv
        It is thread datas from a Discord QA Forum Channel.
        
        It is a list of messages in a discord channel, the first message is the probably question and the rest are probably answers.
        All messages are from the same thread in a forum channel. Read all the messages and infer the question and answer pair.
        Find the correct answer for the question and return it as the final result.
        
        Infer, evaulate and create Full Question: Detailed Answer Pair
        Return a valid JSON as the final result, if there is no answer in the messages, return null. Thank you is not an answer, this data will be used for training so please remove unnecessary data.
        Give me a JSON file with the following format:
        {
        "question": "The question",
        "answer": "The answer" or None
        }
    """,
    tools=[
        {"type": "retrieval"},
        {"type": "code_interpreter"}
    ],
    model="gpt-4-1106-preview"
)

In [25]:
thread = client.beta.threads.retrieve(
    thread_id="thread_HqgyK97QxH7CTFTXVUAc6ZSt"
)

In [26]:
runs = client.beta.threads.runs.list(
    thread_id="thread_HqgyK97QxH7CTFTXVUAc6ZSt"
)

In [32]:
messagess = client.beta.threads.messages.list(
    thread_id="thread_HqgyK97QxH7CTFTXVUAc6ZSt"
)

In [46]:
messagess.data[2].content[0].text.annotations[0].file_path.file_id

'file-EoYS3XtFLzMAB41IBsKwO8Sr'

In [47]:
file_output = client.files.retrieve(
    file_id=messagess.data[2].content[0].text.annotations[0].file_path.file_id
)
file_output

FileObject(id='file-EoYS3XtFLzMAB41IBsKwO8Sr', bytes=228, created_at=1699827917, filename='/mnt/data/qa_pair.json', object='file', purpose='assistants_output', status='processed', status_details=None)

In [50]:
content = client.files.content(file_id=messagess.data[2].content[0].text.annotations[0].file_path.file_id)
content.content

b'{"question": "What commitment scheme is most efficient to use in a zkApp?", "answer": "Poseidon hash, as using this with a custom gate to create commitments is more efficient than doing scalar multiplications in provable code."}'

In [9]:
file_path

'processed1.txt'

In [21]:
file = client.files.create(
  file=open(file_path, "rb"),
  purpose="assistants"
)

In [22]:
assistant_file = client.beta.assistants.files.create(
  assistant_id=assistant.id, 
  file_id=file.id
)

In [23]:
thread = client.beta.threads.create()

In [14]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create Full Question: Detailed Answer pair"
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
print("Run creation status: ", run.status)
while True:
   run_object = client.beta.threads.runs.retrieve(
     thread_id=thread.id,
     run_id=run.id
   )
   if run_object.status == 'completed':
      break
   if run_object.status == 'failed' or run_object.status == 'requires_action':
      print(run_object.model_dump()["last_error"]["message"])
   print("Runned but not completed yet, waiting 5 seconds", run_object.status)
   time.sleep(2)
runned = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages)

Run creation status:  queued
Runned but not completed yet, waiting 5 seconds in_progress
Runned but not completed yet, waiting 5 seconds in_progress
Runned but not completed yet, waiting 5 seconds in_progress
Runned but not completed yet, waiting 5 seconds in_progress
Runned but not completed yet, waiting 5 seconds in_progress
Runned but not completed yet, waiting 5 seconds in_progress
SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_Tio5TYr4DFO38V4ZXSA4L8C2', assistant_id='asst_bkzbibx63TvejcFhRva0QzvM', content=[MessageContentText(text=Text(annotations=[], value='To create a full question and detailed answer pair from the thread data provided, it\'s essential to understand the context of the discussion. After reviewing the conversation, it\'s apparent that the topic revolves around setting up blockchain nodes and confirming their readiness.\n\nThe initial inquiry seems to be missing, but based on the responses, I can infer that the original question was about the status of b

In [15]:
print(run_object.model_dump_json())

{"id":"run_zAp4glMwVlYcsu4zbGYXSlna","assistant_id":"asst_bkzbibx63TvejcFhRva0QzvM","cancelled_at":null,"completed_at":1699826868,"created_at":1699826856,"expires_at":null,"failed_at":null,"file_ids":["file-Kumps46N8PbJmvL88xdZ6EkI","file-G1x1RvDHaVHC0v22wey6B3Ti"],"instructions":"\n        You have csv file named processed1.csv\n        It is thread datas from a Discord QA Forum Channel.\n        \n        It is a list of messages in a discord channel, the first message is the probably question and the rest are probably answers.\n        All messages are from the same thread in a forum channel. Read all the messages and infer the question and answer pair.\n        \n        Infer, evaulate and create Full Question: Detailed Answer Pair\n        Return a valid JSON as the final result, if there is no answer in the messages, return null. Thank you is not an answer, this data will be used for training so please remove unnecessary data.\n        {\n        \"question\": \"The question\",\

In [17]:
for i, message in enumerate(messages.data):
  print(f'Message {i+1}:')
  print(message.content[0].text.value)

Message 1:
To create a full question and detailed answer pair from the thread data provided, it's essential to understand the context of the discussion. After reviewing the conversation, it's apparent that the topic revolves around setting up blockchain nodes and confirming their readiness.

The initial inquiry seems to be missing, but based on the responses, I can infer that the original question was about the status of blockchain node (bp) setup for a project, as acknowledged by one participant's confirmation and another's mention of an email checker and nodes being set up.

Given the responses, the most logical inferred question and answer pair would be:

**Question:** "Has everyone set up their blockchain nodes as per the instructions, and can anyone report if they're running smoothly or experiencing issues?"

**Answer:** Multiple members confirmed that their nodes are set up and running smoothly. One participant mentioned not to trust email trackers as their email actively kills t

In [39]:
text = messages.data[0].content[0].text.value
question_index = text.find("**Full Question:**")
answer_index = text.find("**Detailed Answer:**")

full_question = text[question_index + len("**Full Question:**"):answer_index].strip()
detailed_answer = text[answer_index + len("**Detailed Answer:**"):].strip()

result_json = {
    "FullQuestion": full_question,
    "DetailedAnswer": detailed_answer
}

print(json.dumps(result_json, indent=2))

{
  "FullQuestion": "n you guys post the address list of nodes that can't be accessed via this port so we may check on our own again to ensure?\n\nDetailed Answer: Thanks for this suggestion. I have published the list",
  "DetailedAnswer": "you guys post the address list of nodes that can't be accessed via this port so we may check on our own again to ensure?\n\nDetailed Answer: Thanks for this suggestion. I have published the list."
}
